In [ ]:
# takes an array of dict_data
# and combines embeddings by averaging with self then appending with other
# convers to array with labels and returns
def ensemble_append_mean_reps(dicts, labels):
    
    new_dict = dict()
    keys = dicts[0].keys()
    for key in keys:
        seqs = []
        for d in dicts:
            seq = np.mean(d[key], axis=0)
            seqs.append(seq)
        combined_seqs = np.concatenate(seqs, axis=0)
        new_dict[key] = combined_seqs
        
    emb_size = list(new_dict.values())[0].shape[0]
    X = np.zeros((len(new_dict), emb_size))
    y = np.zeros(len(new_dict))
    
    i = 0
    for key in new_dict:
        X[i] = np.mean(new_dict[key], axis=0)
        y[i] = labels[key]
        i += 1
        
    return X, y

In [ ]:
task = 'remote_homology'

X_train_e = read_dataset('elmo', task, 'train')
X_train_u = read_dataset('unirep', task, 'train')
y_train_dict = read_dataset('label', task, 'train')

In [ ]:
X_train_app, y_train = ensemble_append_mean_reps([X_train_e, X_train_u], y_train_dict)
print(X_train_app.shape)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

clf_app = OneVsRestClassifier(LinearSVC())
clf_app.fit(X_train_app, y_train)

In [ ]:
X_valid_e = read_dataset('elmo', task, 'valid')
X_valid_u = read_dataset('unirep', task, 'valid')
y_valid_dict = read_dataset('label', task, 'valid')
X_valid_app, y_valid = ensemble_append_mean_reps([X_valid_e, X_valid_u], y_valid_dict)

valid_score = clf_app.score(X_valid_app, y_valid)
print(f"append model valid score: ", valid_score)

In [ ]:
X_test_e = read_dataset('elmo', task, 'test')
X_test_u = read_dataset('unirep', task, 'test')
y_test_dict = read_dataset('label', task, 'test')
X_test_app, y_test = ensemble_append_mean_reps([X_test_e, X_test_u], y_test_dict)

test_score = clf_app.score(X_test_app, y_test)
print(f"append model test score: ", test_score)